The first approach is similar to the one used by C19. We fitted the
RV data using Keplerians for the planetary signals and a GP with
a quasi-periodic kernel for the stellar activity. The mathematical
expression of the kernel of this GP is

$$
GP(t_i, t_j) = \eta_1^2 exp[-\frac{|t_i-t_j|^2}{\eta_2^2} -\frac{sin^2(\pi|t_i-t_j|/\eta_3)}{2\eta_4^2}] 
$$

$$
\eta_1 = \text{amplitude of the covariance; Arv}
$$
$$
\eta_2 = \text{decay timescale; } \tau_{\text{decay}}
$$
$$
\eta_3 = \text{period of recurrence of the covariance; } P_{\text{rot}}
$$
$$
\eta_4 = \text{periodic coherence scale; } \gamma
$$


where Arv is the amplitude of the covariance, τdecay is the decay timescale, Prot is the period of recurrence of the covariance,
and γ is the periodic coherence scale (e.g., Grunblatt et al. 2015).
We used the Python package george